### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [47]:
import numpy as np
import pandas as pd

In [48]:
# Read in the datasets
movies = pd.read_csv('../data/movies_clean.csv', index_col=0)
reviews = pd.read_csv('../data/reviews_clean.csv', index_col=0)

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [49]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05 21:00:50
1,2,208092,5,1586466072,2020-04-09 21:01:12
2,2,358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [50]:
reviews['date'] = pd.to_datetime(reviews['date'])

In [51]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100001 entries, 0 to 100000
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   user_id    100001 non-null  int64         
 1   movie_id   100001 non-null  int64         
 2   rating     100001 non-null  int64         
 3   timestamp  100001 non-null  int64         
 4   date       100001 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 4.6 MB


In [52]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    reviews = reviews.sort_values(by=order_by)
    training_df = reviews.iloc[:training_size]
    validation_df = reviews.iloc[training_size:(training_size+testing_size)]
    
    return training_df, validation_df

In [53]:
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [54]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-07-19', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-08-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 300 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.

In [55]:
# my own Funk SVD
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.sum(~np.isnan(ratings_mat))# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features) # user matrix filled with random values of shape user x latent
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    errors = []

    for i in range(iters):
        # for each iteration
        # keep track of iteration and MSE
        print("Optimizaiton Statistics")
        print(f"Iterations {i} | Mean Squared Error {np.mean(errors)}")

        # update our sse
        sse_accum = 0
        errors = []
        
        # For each user-movie pair
        for user in range(n_users):
            # if the rating exists
            ratings_index = np.argwhere(~np.isnan(ratings_mat[user]))[:,1]
            for movie in ratings_index:

                actual = ratings_mat[user,movie]
                pred = np.dot(user_mat[user,:], movie_mat[:,movie])
 
                # compute the error as the actual minus the dot product of the user and movie latent features
                error = actual - pred
                # Keep track of the sum of squared errors for the matrix
                errors.append(error**2)
                sse_accum += error**2
                
                for latent_feature in range(latent_features):

                    u_old = user_mat[user,latent_feature]
                    v_old = movie_mat[latent_feature,movie]
                    # update the values in each matrix in the direction of the gradient
                    u_new = u_old + (learning_rate * 2 * (actual - pred) * v_old)
                    v_new = v_old + (learning_rate * 2 * (actual - pred) * u_old)
                    
                    user_mat[user,latent_feature] = u_new
                    movie_mat[latent_feature,movie] = v_new



        # print results
        
    return user_mat, movie_mat 

In [56]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.matrix(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, 
                              latent_features=15, 
                              learning_rate=0.005, 
                              iters=300)

Optimizaiton Statistics
Iterations 0 | Mean Squared Error nan


c:\Users\Pontello\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Pontello\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Optimizaiton Statistics
Iterations 1 | Mean Squared Error 9.342714204211672
Optimizaiton Statistics
Iterations 2 | Mean Squared Error 5.069018127897194
Optimizaiton Statistics
Iterations 3 | Mean Squared Error 3.707843602637218
Optimizaiton Statistics
Iterations 4 | Mean Squared Error 2.918092625054307
Optimizaiton Statistics
Iterations 5 | Mean Squared Error 2.381116326650534
Optimizaiton Statistics
Iterations 6 | Mean Squared Error 1.9848902536966542
Optimizaiton Statistics
Iterations 7 | Mean Squared Error 1.6781418968517348
Optimizaiton Statistics
Iterations 8 | Mean Squared Error 1.4333688193875425
Optimizaiton Statistics
Iterations 9 | Mean Squared Error 1.234109886774717
Optimizaiton Statistics
Iterations 10 | Mean Squared Error 1.0696589731522683
Optimizaiton Statistics
Iterations 11 | Mean Squared Error 0.9325564081929656
Optimizaiton Statistics
Iterations 12 | Mean Squared Error 0.817323166789266
Optimizaiton Statistics
Iterations 13 | Mean Squared Error 0.7197966542408494
Op

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [57]:
user_mat

array([[0.9800176 , 0.70041997, 0.87741446, ..., 0.56170305, 1.30646954,
        0.96370461],
       [1.04504599, 0.32102724, 0.32063146, ..., 0.95969245, 1.1846524 ,
        1.08421485],
       [0.57872853, 0.54508325, 0.34978686, ..., 1.10858076, 0.66674048,
        0.99248102],
       ...,
       [1.16873858, 1.0796369 , 0.60997013, ..., 0.90909978, 0.95683815,
        0.90740968],
       [0.56680838, 1.18109344, 0.50476521, ..., 0.14201267, 1.17253179,
        1.26471532],
       [0.79633683, 0.55390426, 0.62432979, ..., 0.38933057, 0.19829808,
        0.56701753]])

In [58]:
train_data_df.index

Int64Index([   4,   17,   21,   22,   26,   29,   32,   37,   43,   45,
            ...
            7963, 7976, 7977, 7993, 7998, 8000, 8001, 8006, 8007, 8008],
           dtype='int64', name='user_id', length=1542)

In [70]:
int(np.argwhere(train_data_df.index == 2625))

486

In [78]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Create series of users and movies in the right order
    user = np.argwhere(train_data_df.index == user_id)[0][0]
    movie = np.argwhere(train_data_df.columns == movie_id)[0][0]
    
    # User row and Movie Column
    
    # Take dot product of that row and column in U and V to make prediction
    predictions = np.dot(user_matrix,movie_matrix)

    pred = predictions[user,movie]
    
    return pred

In [63]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 2625, 169547)
pred_val

8.662601075542433

In [64]:
train_data_df[169547].loc[2625]

nan

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [65]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Documentary,Adventure,Animation,...,War,Romance,Musical,Film-Noir,Sport,Mystery,Western,Horror,Comedy,Music
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [66]:
movies['movie'][movies['movie_id']==169547].values[0]

'American Beauty (1999)'

In [67]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    '''
    movie_name = movies['movie'][movies['movie_id']==movie_id].values[0] # Implement your code here

    print("For user {} we predict a {} rating for the movie {}.".format(user_id, round(prediction, 2), str(movie_name)))

In [68]:
# Test your function the the results of the previous function
print_prediction_summary(2625, 169547, pred_val)

For user 2625 we predict a 8.66 rating for the movie American Beauty (1999).


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [83]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    val_users = np.array(val_df['user_id'])
    val_movies = np.array(val_df['movie_id'])
    val_ratings = np.array(val_df['rating'])
    
    for idx in range(num_preds):
        try:
            pred = predict_rating(user_mat, movie_mat, val_users[idx], val_movies[idx])
            movie_name = movies['movie'][movies['movie_id']==val_movies[idx]].values[0]
            print("The actual rating for user {} on movie {} is {}.\n While the predicted rating is {}.".format(val_users[idx], movie_name, val_ratings[idx], round(pred))) 
        except IndexError:
            continue
# Perform the predicted vs. actual for the first 10 rows.  How does it look?
validation_comparison(val_df, 10)

The actual rating for user 6410 on movie This Is the End (2013) is 8.
 While the predicted rating is 7.
The actual rating for user 2894 on movie Only God Forgives (2013) is 3.
 While the predicted rating is 6.
The actual rating for user 2625 on movie American Beauty (1999) is 10.
 While the predicted rating is 9.
The actual rating for user 5490 on movie Man of Steel (2013) is 6.
 While the predicted rating is 10.
